#Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import warnings 
warnings.filterwarnings('ignore')

#Load train data

In [2]:
# Read 'train_new.csv' file
train_data=pd.read_csv('train_new.csv',index_col='id')
train_data.head()

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,theme,crawl_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
272,8.00,7,EUR,10000,1,4.0,1670,0,0,0,0,"Summer,soildcolor,Plus Size,Tank,camisole,Tops...",yellow,M,50,Livraison standard,3,0,30,50,NaN,NaN,CN,keepahorse,keepahorse,"88 % avis positifs (66,644 notes)",66644,4.137582,577fb2b368116418674befd9,1,https://s3-us-west-1.amazonaws.com/sweeper-pro...,summer,2020-08
1515,8.00,7,EUR,1000,0,4.0,843,1,1,0,0,"bathing suit,Plus Size,bikini set,sexy swimsui...",black,L,50,Livraison standard,3,0,37,50,NaN,NaN,CN,shanghaimingqidianqikejiyouxiangongsi,上海铭启电气科技有限公司,"91 % avis positifs (25,752 notes)",25752,4.256873,566a3ef17233ff2686443082,0,NaN,summer,2020-08
940,6.00,34,EUR,100,0,4.0,10,0,0,0,0,"Summer,Vest,momshirt,Get,summer t-shirts,funny...",white,L,50,Livraison standard,2,0,41,50,NaN,NaN,CN,zhaodongmei,zhaodongmei,"83 % avis positifs (3,897 notes)",3897,3.928920,5aacd6c1ccf0c83e5a6f3de0,0,NaN,summer,2020-08
309,4.67,4,EUR,100,1,3.0,82,0,0,0,0,"Summer,Shorts,pants,Beach,Plus Size,beachpant,...",lakeblue,XS,5,Livraison standard,1,0,25,50,NaN,NaN,CN,pookie0331,pookie0331,"87 % avis positifs (7,497 notes)",7497,4.079365,583141fbfef4094e51453d9b,0,NaN,summer,2020-08
226,8.00,11,EUR,1000,0,4.0,127,0,0,0,0,"Summer,Floral print,women dresses,fashion dres...",apricot,S,50,Livraison standard,3,0,15,50,1.0,Quantité limitée !,CN,shitongyi12683626,shitongyi12683626,91 % avis positifs (173 notes),173,4.306358,5d3bc9f8e13a7e654424a4cb,0,NaN,summer,2020-08


#Preprocessing Train Data




our data has some noise and mising values which have bad effect in our model performance .so i will make preprocessing on this data to try to get high performance 

In [3]:
# Show number of mising data in each column
train_data.isnull().sum()

price                             0
retail_price                      0
currency_buyer                    0
units_sold                        0
uses_ad_boosts                    0
rating                            0
rating_count                      0
badges_count                      0
badge_local_product               0
badge_product_quality             0
badge_fast_shipping               0
tags                              0
product_color                    29
product_variation_size_id        12
product_variation_inventory       0
shipping_option_name              0
shipping_option_price             0
shipping_is_express               0
countries_shipped_to              0
inventory_total                   0
has_urgency_banner              745
urgency_text                    745
origin_country                   12
merchant_title                    0
merchant_name                     4
merchant_info_subtitle            0
merchant_rating_count             0
merchant_rating             

**Drop columns**

'merchant_name' and 'merchant_title' columns  have almost the same data So I droped'merchant_name' column

'inventory_total' column has a very little number of data, so I drop it 
'currency_buyer', 'theme', and 'crawl_month' columns that have the same data all over the rows, so I dropped these columns 

'badge_fast_shipping','shipping_is_express','urgency_text','merchant_name','merchant_profile_picture' columns that may have no effect on the product's rate,so I dropped them on train data




In [4]:
# to count each uniqe in column
train_data['inventory_total'].value_counts()

50    1089
1        1
9        1
37       1
2        1
36       1
Name: inventory_total, dtype: int64

In [5]:
# Drop ('currency_buyer','badge_fast_shipping','shipping_is_express',
#       'inventory_total','urgency_text','merchant_name','merchant_profile_picture',
#        'merchant_info_subtitle','merchant_id','crawl_month','theme') columns that may have no impacts in the products rating 
        
train_data=train_data.drop(columns=['currency_buyer','badge_fast_shipping','shipping_is_express',
                                    'inventory_total','urgency_text','merchant_name','merchant_profile_picture',
                                    'merchant_info_subtitle','merchant_id','crawl_month','theme'])                   

**Drop rows**

Becouse of  the Ratings are in categories from 1 to 5 and I found only one row that has rating=6, so i will drop this row 
in general I don't prefer to drop rows because it may affect our data but here it is only one row


In [6]:
# To show the unique value in (rating) column and how many each uniqe is been reppeated
train_data.loc[:, 'rating'].unique()
train_data.loc[:, 'rating'].value_counts()

4.0    774
5.0    170
3.0    135
2.0     12
1.0      2
6.0      1
Name: rating, dtype: int64

In [47]:
# drop row that has rating=6
train_data = train_data.drop(train_data[train_data.rating == 6].index)

**Data imputation**

there are columns that have mising values So I fill columns that have categorical data with (other) like ('product_color','product_variation_size_id','origin_country) columns

has_urgency_banner' column must be filled with 0.because  there is text in the 'urgency text' column, each row of this column equals one. As a result so the empty rows in the 'has urgency banner' column must be filled with 0. 
on train data


In [8]:
# Fill mising value in (has_urgency_banner) column with zero
# Fill mising value in (product_color) column with other
# Fill mising value in (product_variation_size_id)column with other
# Fill mising value in (origin_country)column with other

train_data['has_urgency_banner'] = train_data['has_urgency_banner'].fillna(0)
train_data['product_variation_size_id'] = train_data['product_variation_size_id'].fillna('other')
train_data['product_color'] = train_data['product_color'].fillna('other')
train_data['origin_country'] = train_data['origin_country'].fillna('other')

In [9]:
# To find how many uniqe variable in (product_color) column and count how many each uniqe
train_data.loc[:, 'product_color'].unique()
train_data.loc[:, 'product_color'].value_counts()

black               219
white               173
blue                 77
yellow               69
pink                 68
                   ... 
claret                1
brown & yellow        1
violet                1
Pink                  1
navyblue & white      1
Name: product_color, Length: 87, dtype: int64

**preprocessing (product_color)** 

Handling variable in (product_color) column by collecting all the same colors but have different names to replace them by the common name
(ex.)'watermelonred','rosered','lightred' replace to red in train data

In [10]:
# preprocessing (product_color) colmn (handling its values) 

train_data.replace(to_replace= ['white','White','whitefloral','offwhite','ivory'], value = 'white', inplace=True)

train_data.replace(to_replace= ['blue','Blue','lakeblue','lightblue','skyblue','darkblue','denimblue'], value = 'blue')
train_data.replace(to_replace= ['navy','navy blue'], value = 'navyblue', inplace=True)

train_data.replace(to_replace= ['darkgreen','fluorescentgreen','mintgreen','applegreen','lightgreen','light green'], value = 'green')
train_data.replace(to_replace= ['Army green'], value = 'armygreen', inplace=True)

train_data.replace(to_replace= ['red','RED','watermelonred','rosered','lightred','burgundy','claret'], value = 'red')
train_data.replace(to_replace= ['wine red','wine'], value = 'winered', inplace=True)

train_data.replace(to_replace= ['black','Black','coolblack'], value = 'black', inplace=True )

train_data.replace(to_replace= ['pink','Pink','lightpink','rose','dustypink','rosegold','dustypink'], value = 'pink',inplace=True)

train_data.replace(to_replace= ['army','leopard','white & green','pink & black','pink & white','white & green','black & yellow',
                                'star','camouflage','whitestripe','navyblue & white',
                                'black & blue','pink & blue','rainbow','floral','brown & yellow',
                                'blue & pink','orange-red','winered & yellow','blackwhite'], value = 'other_colores',inplace=True)

train_data.replace(to_replace= ['grey','lightgrey'], value = 'gray', inplace=True)

train_data.replace(to_replace= ['lightyellow','gold','camel'], value = 'yellow',inplace=True)

column_values = train_data[["product_color"]].values.ravel()


**preprocessing (product_variation_size_id)**

Handling variable in (product_variation_size_id) column by collecting all the same size but have different names to replace them by the common name

 (ex.) ['XS', 'XS.','Size-XS', 'SIZE XS']  repleace to 'XS' and so on and the variables which do not refer to size collect them to variable called (Others) in train data

In [11]:
# preprocessing (product_variation_size_id) colmn (handling its values)

train_data.replace(to_replace= ['S.','S(bust 88cm)', 's', 'Size S', 'Size-S', 'S/M(child)', 'S (waist58-62cm)', 'S Pink','US-S', 
         'pants-S','Size S.', 'S..','25-S', 'size S', 'Size/S', 'Suit-S', 'SIZE S'], value = 'S', inplace=True )
train_data.replace(to_replace= ['XXS','Size-XXS', 'SIZE XXS','Size -XXS', 'Size XXS','SIZE-XXS'], value = '2XS', inplace=True )
train_data.replace(to_replace= ['M','Size M','M.'], value = 'M', inplace=True )
train_data.replace(to_replace= ['L','SizeL'], value = 'L', inplace=True )
train_data.replace(to_replace= ['XS', 'XS.','Size-XS', 'SIZE XS'], value = 'XS', inplace=True )
train_data.replace(to_replace= ['XL', 'X   L', '1 PC - XL'], value = 'XL', inplace=True )
train_data.replace(to_replace= ['XXL', '2XL'], value = '2XL', inplace=True )
train_data.replace(to_replace= ['4XL','XXXXL','Size4XL'], value = '4XL', inplace=True )
train_data.replace(to_replace= ['XXXXXL', 'Size-5XL','5XL'], value = '5XL', inplace=True )
train_data.replace(to_replace= ['XXXS'], value = '3XS', inplace=True )

train_data.replace(to_replace= ['H01','1pc','34','Floating Chair for Kid','60','100 cm','6XL','5PAIRS',
                               '1','100 x 100cm(39.3 x 39.3inch)','Base Coat','choose a size','26(Waist 72cm 28inch)',
                               'Pack of 1','Round','5','10pcs','40 cm','white','1m by 3m ',
                               '4-5 Years','Base & Top & Matte Top Coat','20pcs','B','Baby Float Boat',
                               'One Size','30 cm','20PCS-10PAIRS','2','2pcs','25','29','33',
                               '35','10 ml','Women Size 36','1m by 3m','1 pc.','3XL','04-3XL'], value = 'Others', inplace=True )

size_column = train_data[["product_variation_size_id"]].values.ravel()
unique_value =  pd.unique(size_column)
unique_value

array(['M', 'L', 'XS', 'S', 'XL', 'Others', '2XS', 'other', '3XS', '2XL',
       '4XL', '5XL'], dtype=object)

In [12]:
# To find how many uniqe variable in (shipping_option_name) column and count how many each uniqe
train_data.loc[:, 'shipping_option_name'].unique()
train_data.loc[:, 'shipping_option_name'].value_counts()

Livraison standard         1047
Standard Shipping            16
Envio Padrão                  6
الشحن القياسي                 4
Expediere Standard            4
Envío normal                  3
Standardversand               3
Standardowa wysyłka           2
Standart Gönderi              2
Livraison Express             2
Стандартная доставка          2
Spedizione standard           1
ការដឹកជញ្ជូនតាមស្តង់ដារ       1
Name: shipping_option_name, dtype: int64

**shipping_is_express column**

Handling variable in (shipping_is_express) column by replacing the variable ['Standard Shipping','الشحن القياسي','Expediere Standard','Standardversand','Standardowa wysyłka',
'Standart Gönderi','Стандартная доставка','Spedizione standard','ការដឹកជញ្ជូនតាមស្តង់ដារ'] that have the same meaning with 'Livraison standard' in training  

In [13]:
# preprocessing (shipping_option_name) colmn (handling its values)
#Handling variable in (shipping_is_express) column by replacing the variable ['Standard Shipping','الشحن القياسي','Expediere Standard','Standardversand','Standardowa wysyłka',
#'Standart Gönderi','Стандартная доставка','Spedizione standard','ការដឹកជញ្ជូនតាមស្តង់ដារ'] that have the same meaning with 'Livraison standard' in training & testing data 

train_data.replace(to_replace= ['Standard Shipping','الشحن القياسي','Expediere Standard',
                               'Standardversand','Standardowa wysyłka','Standart Gönderi',
                               'Стандартная доставка','Spedizione standard',
                                'ការដឹកជញ្ជូនតាមស្តង់ដារ'], value = 'Livraison standard', inplace=True )

In [14]:
# To ensure that there is no mising data in columns 
train_data.isnull().sum()

price                           0
retail_price                    0
units_sold                      0
uses_ad_boosts                  0
rating                          0
rating_count                    0
badges_count                    0
badge_local_product             0
badge_product_quality           0
tags                            0
product_color                   0
product_variation_size_id       0
product_variation_inventory     0
shipping_option_name            0
shipping_option_price           0
countries_shipped_to            0
has_urgency_banner              0
origin_country                  0
merchant_title                  0
merchant_rating_count           0
merchant_rating                 0
merchant_has_profile_picture    0
dtype: int64

In [15]:
# find out all data Type
# To prints information about the train data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1093 entries, 272 to 1536
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price                         1093 non-null   float64
 1   retail_price                  1093 non-null   int64  
 2   units_sold                    1093 non-null   int64  
 3   uses_ad_boosts                1093 non-null   int64  
 4   rating                        1093 non-null   float64
 5   rating_count                  1093 non-null   int64  
 6   badges_count                  1093 non-null   int64  
 7   badge_local_product           1093 non-null   int64  
 8   badge_product_quality         1093 non-null   int64  
 9   tags                          1093 non-null   object 
 10  product_color                 1093 non-null   object 
 11  product_variation_size_id     1093 non-null   object 
 12  product_variation_inventory   1093 non-null   int64  
 13  s

**categorical data**

All input and output variables in machine learning models must be numeric, which means that I must convert categorical data to numbers before fitting and evaluating a model wherefore I used label encoder to convert all categorical data to numerical

In [16]:
# To convert all categorical data into numerical data (Train data)
el = LabelEncoder()
x_coulmuns = ['tags','product_color','product_variation_size_id','shipping_option_name','origin_country','merchant_title']
for i in x_coulmuns:
   train_data.loc[:, i] = el.fit_transform(train_data[i])

#Load test data

In [17]:
# read 'test_new.csv' file
test_data=pd.read_csv('test_new.csv',index_col='id')
test_data.head()

,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,tags,product_color,product_variation_size_id,product_variation_inventory,shipping_option_name,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,urgency_text,origin_country,merchant_title,merchant_name,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,theme,crawl_month
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
937,5.75,5,EUR,10,1,0,0,0,0,0,"Funny,letter print,Fashion,Shirt,Sleeve,Summer...",white,XS,5,Livraison standard,2,0,139,50,NaN,NaN,CN,liyang163,liyang163,33 % avis positifs (3 notes),3,2.333333,5e63469b2fdc774466e15dd5,0,NaN,summer,2020-08
270,7.00,7,EUR,5000,0,579,0,0,0,0,"Women Vest,Vest,Fashion,Women Blouse,long dres...",black,XS,50,Livraison standard,2,0,31,50,NaN,NaN,CN,singing the song,singingthesong,"86 % avis positifs (4,213 notes)",4213,4.058391,583547f417dc6224cb464117,0,NaN,summer,2020-08
308,5.00,16,EUR,1000,1,62,0,0,0,0,"hollowouttanktop,Tanktops for women,Plus Size,...",blue,XS,20,Livraison standard,2,0,41,50,NaN,NaN,CN,Surper boutique,surperboutique,"85 % avis positifs (10,501 notes)",10501,4.029235,57086fd7fa6bee434d0e5852,0,NaN,summer,2020-08
913,18.00,27,EUR,5000,1,1903,0,0,0,0,"men jeans,Shorts,Summer,Vintage,Denim,Men,casu...",blue,M,50,Livraison standard,5,0,42,50,NaN,NaN,CN,postonny,postonny,"86 % avis positifs (15,855 notes)",15855,4.052917,5a17b55d1f4d8c4f0b44b0ce,0,NaN,summer,2020-08
403,12.00,81,EUR,20000,0,1799,0,0,0,0,"Shorts,Lace,pants,Dress,Women Leggings,Hot pan...",white,M,50,Livraison standard,3,0,38,50,NaN,NaN,CN,fashionforgirls,guangzhouchanny,"88 % avis positifs (151,914 notes)",151914,4.127921,53aa664438d3046ee44a5024,0,NaN,summer,2020-08


#Preprocessing Test Data

As on train data our test data  has some noise and mising values which have bad effect in our model performance .so i will make preprocessing on this data to try to get high performance

In [18]:
#to know which colimns that have mising values
test_data.isnull().sum()

price                             0
retail_price                      0
currency_buyer                    0
units_sold                        0
uses_ad_boosts                    0
rating_count                      0
badges_count                      0
badge_local_product               0
badge_product_quality             0
badge_fast_shipping               0
tags                              0
product_color                    12
product_variation_size_id         2
product_variation_inventory       0
shipping_option_name              0
shipping_option_price             0
shipping_is_express               0
countries_shipped_to              0
inventory_total                   0
has_urgency_banner              355
urgency_text                    355
origin_country                    5
merchant_title                    0
merchant_name                     0
merchant_info_subtitle            1
merchant_rating_count             0
merchant_rating                   0
merchant_id                 

**Drop columns**

'merchant_name' and 'merchant_title' columns  have almost the same data So I droped'merchant_name' column

'inventory_total' column has a very little number of data, so I drop it 
'currency_buyer', 'theme', and 'crawl_month' columns that have the same data all over the rows, so I dropped these columns 

'badge_fast_shipping','shipping_is_express','urgency_text','merchant_name','merchant_profile_picture' columns that may have no effect on the product's rate,so I dropped them on test data

In [19]:
# Drop (currency_buyer','badge_fast_shipping','shipping_is_express',
#                                    'inventory_total','urgency_text','merchant_name','merchant_profile_picture',
#                                    'merchant_info_subtitle','merchant_id','crawl_month','theme)column

test_data=test_data.drop(columns=['currency_buyer','badge_fast_shipping','shipping_is_express',
                                    'inventory_total','urgency_text','merchant_name','merchant_profile_picture',
                                    'merchant_info_subtitle','merchant_id','crawl_month','theme'])

**Data imputation**

there are columns that have mising values So I fill columns that have categorical data with (other) like ('product_color','product_variation_size_id','origin_country) columns

has_urgency_banner' column must be filled with 0.because  there is text in the 'urgency text' column, each row of this column equals one. As a result so the empty rows in the 'has urgency banner' column must be filled with 0. 
on test data


In [20]:
# Data imputation
# Fill mising value in (has_urgency_banner) column with zero
# Fill mising value in (product_color) column with ffill
# Fill mising value in (product_variation_size_id)column with other
# Fill mising value in (origin_country)column with other

test_data['has_urgency_banner'] = test_data['has_urgency_banner'].fillna(0)
test_data['product_color'] = test_data['product_color'].fillna('other')
test_data['product_variation_size_id'] = test_data['product_variation_size_id'].fillna('other')
test_data['origin_country'] = test_data['origin_country'].fillna('other_country')

**preprocessing (product_color) column**

Handling variable in (product_color) column by collecting all the same colors but have different names to replace them by the common name
(ex.)'watermelonred','rosered','lightred' replace to red in test data

In [21]:
# preprocessing (product_color) colmn (handling its values)
# Handling variable in (product_color) column by collecting all the same colors but have different names to replace them by the common name
#(ex.)'watermelonred','rosered','lightred' replace to red
test_data.replace(to_replace= ['white','White','whitefloral','offwhite'], value = 'white', inplace=True)

test_data.replace(to_replace= ['blue','Blue','lakeblue','lightblue','skyblue','darkblue','denimblue'], value = 'blue')
test_data.replace(to_replace= ['navy','navy blue'], value = 'navyblue', inplace=True)

test_data.replace(to_replace= ['darkgreen','fluorescentgreen','mintgreen','applegreen','lightgreen','light green'], value = 'green')
test_data.replace(to_replace= ['Army green'], value = 'armygreen', inplace=True)

test_data.replace(to_replace= ['red','RED','watermelonred','rosered','lightred','burgundy','claret','wine'], value = 'red')
test_data.replace(to_replace= ['wine red'], value = 'winered', inplace=True)

test_data.replace(to_replace= ['black','Black','coolblack'], value = 'black', inplace=True )

test_data.replace(to_replace= ['pink','Pink','lightpink','rose','dustypink','rosegold','dustypink'], value = 'pink',inplace=True)

test_data.replace(to_replace= ['army','leopard','white & green','pink & black','pink & white','white & green','black & yellow',
                                'star','camouflage','whitestripe','navyblue & white',
                                'black & blue','pink & blue','rainbow','floral','brown & yellow',
                                'blue & pink','orange-red','winered & yellow','blackwhite'], value = 'other_colores',inplace=True)

test_data.replace(to_replace= ['grey','lightgrey'], value = 'gray', inplace=True)

test_data.replace(to_replace= ['lightyellow','gold','camel'], value = 'yellow',inplace=True)

test_data.replace(to_replace= ['nude','ivory'], value = 'beige',inplace=True)


column_values = test_data[["product_color"]].values.ravel()


**preprocessing (product_variation_size_id)**

Handling variable in (product_variation_size_id) column by collecting all the same size but have different names to replace them by the common name

(ex.) ['XS', 'XS.','Size-XS', 'SIZE XS']  repleace to 'XS' and so on and the variables which do not refer to size collect them to variable called (Others) in test data

In [22]:
# preprocessing (product_variation_size_id) colmn (handling its values)
#Handling variable in (product_variation_size_id) column by collecting all the same size but have different names to replace them by the common name
#(ex.) ['XS', 'XS.','Size-XS', 'SIZE XS']  repleace to 'XS' and so on and the variables which do not refer to size collect them to variable called (Others) in train data

test_data.replace(to_replace= ['S.','S(bust 88cm)', 's', 'Size S', 'Size-S', 'S/M(child)', 'S (waist58-62cm)', 'S Pink','US-S', 
         'pants-S','Size S.', 'S..','25-S', 'size S', 'Size/S', 'Suit-S', 'SIZE S'], value = 'S', inplace=True )
test_data.replace(to_replace= ['XXS','Size-XXS', 'SIZE XXS','Size -XXS', 'Size XXS','SIZE-XXS'], value = '2XS', inplace=True )
test_data.replace(to_replace= ['M','Size M','M.'], value = 'M', inplace=True )
test_data.replace(to_replace= ['L','SizeL'], value = 'L', inplace=True )
test_data.replace(to_replace= ['XS', 'XS.','Size-XS', 'SIZE XS'], value = 'XS', inplace=True )
test_data.replace(to_replace= ['XL', 'X   L', '1 PC - XL'], value = 'XL', inplace=True )
test_data.replace(to_replace= ['XXL', '2XL'], value = '2XL', inplace=True )
test_data.replace(to_replace= ['4XL','XXXXL','Size4XL'], value = '4XL', inplace=True )
test_data.replace(to_replace= ['XXXXXL', 'Size-5XL','5XL'], value = '5XL', inplace=True )
test_data.replace(to_replace= ['XXXS'], value = '3XS', inplace=True )
test_data.replace(to_replace= ['XXXS'], value = '3XS', inplace=True )

test_data.replace(to_replace= ['H01','1pc','34','Floating Chair for Kid','60','100 cm','6XL','5PAIRS',
                               '1','100 x 100cm(39.3 x 39.3inch)','Base Coat','choose a size','26(Waist 72cm 28inch)',
                               'Pack of 1','Round','5','10pcs','40 cm','white','1m by 3m ',
                               '4-5 Years','Base & Top & Matte Top Coat','20pcs','B','Baby Float Boat',
                               'One Size','30 cm','20PCS-10PAIRS','2','2pcs','25','29','33',
                               '35','10 ml','Women Size 36','1m by 3m','1 pc.','3XL','04-3XL'], value = 'Others', inplace=True )

column_values = test_data[["product_variation_size_id"]].values.ravel()

**preprocessing shipping_is_express column**

Handling variable in (shipping_is_express) column by replacing the variable ['Standard Shipping','الشحن القياسي','Expediere Standard','Standardversand','Standardowa wysyłka',
'Standart Gönderi','Стандартная доставка','Spedizione standard','ការដឹកជញ្ជូនតាមស្តង់ដារ'] that have the same meaning with 'Livraison standard' in test data 

In [23]:
# preprocessing (shipping_is_express) colmn (handling its values)
#Handling variable in (shipping_is_express) column by replacing the variable ['Standard Shipping','الشحن القياسي','Expediere Standard','Standardversand','Standardowa wysyłka',
#'Standart Gönderi','Стандартная доставка','Spedizione standard','ការដឹកជញ្ជូនតាមស្តង់ដារ'] that have the same meaning with 'Livraison standard' 

test_data.replace(to_replace= ['Standard Shipping','الشحن القياسي','Expediere Standard',
                               'Standardversand','Standardowa wysyłka','Standart Gönderi',
                               'Стандартная доставка','Spedizione standard',
                                'ការដឹកជញ្ជូនតាមស្តង់ដារ'], value = 'Livraison standard', inplace=True )

In [24]:
# To ensure that there is no mising data in columns in Testing_data
test_data.isnull().sum()

price                           0
retail_price                    0
units_sold                      0
uses_ad_boosts                  0
rating_count                    0
badges_count                    0
badge_local_product             0
badge_product_quality           0
tags                            0
product_color                   0
product_variation_size_id       0
product_variation_inventory     0
shipping_option_name            0
shipping_option_price           0
countries_shipped_to            0
has_urgency_banner              0
origin_country                  0
merchant_title                  0
merchant_rating_count           0
merchant_rating                 0
merchant_has_profile_picture    0
dtype: int64

categorical data

All input and output variables in machine learning models must be numeric, which means that I must convert categorical data to numbers before fitting and evaluating a model wherefore I used label encoder to convert all categorical data to numerical

In [25]:
# To convert all categorical data into numerical data (Test data)

el = LabelEncoder()
x_coulmuns = ['tags','product_color','product_variation_size_id','shipping_option_name','origin_country','merchant_title']
for i in x_coulmuns:
    test_data.loc[:, i] = el.fit_transform(test_data[i])


#Split Data 

**rating column**

I moved (rating) column to last position to make it easier when split data into train and test

In [26]:
#move rating column to the last column to make data spliting easiar

reorder_column = train_data.pop('rating')
train_data.insert(len(train_data. columns), 'rating', reorder_column)

In [27]:
# set all features in train data except (rating column) in (X) variable
# set rating column in variable(y)
X=train_data.iloc[:,:-1]
y=train_data.iloc[:,-1]

**The experimental protocol**

The experimental protocol I used is hold out method (train_test_split) function 

using train_test_split to split train data into (X_train, X_test, y_train, y_test) to fit the models and make a prediction

train=0.85 , test=0.15

In [28]:
# split train data into (X_train, X_test, y_train, y_test) to fit the models and make a prediction

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.15, random_state=50)

# Decision Tree  Model 1



**Because of its resistance to noise, tolerance for missing information, management of irrelevant,redundant predictive attribute values,and low computational cost,So decision Tree is one of the most extensively used Machine Learning Algorithms.**

My question is how to tune a Decision Tree. What should be the range of values I should try for the maximum depth, which criteria shoulld I use ("gini","entropy")? which  strategy  should use to choose the split at each node.(“best” or “random” )?

In [29]:
# Decision Tree Classifier without using hyperparametar

#import DecisionTreeClassifier from sklearn
from sklearn.tree import DecisionTreeClassifier

# define decission tree model
decission_tree=DecisionTreeClassifier()
# fit model with train data
decission_tree.fit(X_train, y_train)
# make prediction on training data to know how much the model learn from train data
y_hat2=decission_tree.predict(X_train)
# predict test part from (train data)
y_hat=decission_tree.predict(X_test)

# to check if there is overfitting or no)
print("prediction on training data :",f1_score(y_train, y_hat2, average='micro'))
print("prediction on test part from training data :",f1_score(y_test, y_hat, average='micro'))

# fit all train data
decission_tree.fit(X,y)
# make prediction using test file to make our prediction '

y_hat=decission_tree.predict(test_data)

prediction on training data : 0.9989235737351991
prediction on test part from training data : 0.6707317073170732


In [30]:
# count how many(1,2,3,4,5) in (y_hat)
un, co = np.unique(y_hat, return_counts= True)
print(un, co )

[1. 2. 3. 4. 5.] [  2   1  48 369  59]


In [31]:
# to record the result on file to upload on Kaggle
pred_df = pd.DataFrame(data={'id': np.asarray(test_data.index), 'rating': y_hat})
pred_df.to_csv('pred_1.csv', index=False)

when  I let max depth(defoult ) of the tree , the nodes are expanded until all leaves are pure ,The deeper we allow the tree to grow, the more complex the model will become because we will have more splits and it captures more information about the data and this is one of the reason of overfitting in decision trees because the model will fit perfectly for the training data and will not be able to generalize well on test set. 

in my result I found that the (f1_score) on training data is about 99.8% but on tasting part of data is low about 67% this mean the model did not learn well .
when upload 'pred_1.csv' file my expectation is not very high (f1_score)

**here** when I make the predicition using test file and upload the file on Kaggle, The result was that (f1_score)= 0.72384 


#Decision Tree Model 2

In [32]:
# Decision Tree Classifier with using hyperparametar(max_depth)

#import DecisionTreeClassifier from sklearn
from sklearn.tree import DecisionTreeClassifier

# define decission tree model using hyperparamter
decission_tree_=DecisionTreeClassifier(max_depth=3)
# fit model with train data
decission_tree_.fit(X_train, y_train)
# make prediction on training data to know how much the model learn from train data 
y_hat2=decission_tree_.predict(X_train)
# predict test part from (train data)
y_hat=decission_tree_.predict(X_test)

# to check if there is overfitting or no)
print("prediction on training data :",f1_score(y_train, y_hat2, average='micro'))
print("prediction on test part from training data :",f1_score(y_test, y_hat, average='micro'))
# fit all train data
decission_tree_.fit(X,y)
# make prediction using test file to make our prediction 
y_hat=decission_tree_.predict(test_data)
y_hat

prediction on training data : 0.7825618945102261
prediction on test part from training data : 0.7560975609756099


array([5., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4.,
       5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 5., 4., 5., 5., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 4., 4.

In [34]:
# to record the result on file to upload on Kaggle
pred_df = pd.DataFrame(data={'id': np.asarray(test_data.index), 'rating': y_hat})
pred_df.to_csv('pred_2.csv', index=False)

In the previous model when i did not define (max_depth) the model choose the (defoult) whish is the maximum max_depth  that lead to make the model overfitted  ,To avoid this  overfitting, we should reduce the number for max_depth , So I let (max_depth=3) and I found that the model's (f1_score) on training data is not very high this mean that the model did not learn well and (f1_score) on testing part also low
and gives me not very high fF1_score)on kaggle.it was 0.78242

#Decision Tree Model 3

In [35]:
#Decision Tree Classifier without using hyperparametar

#import DecisionTreeClassifier from sklearn
from sklearn.tree import DecisionTreeClassifier
# define decission tree model
decission_tree=DecisionTreeClassifier(max_depth=6,criterion='entropy',splitter='best')

#fit model with train data
decission_tree.fit(X_train, y_train)

# make prediction on training data to know how much the model learn from train data
y_hat2=decission_tree.predict(X_train)
# predict test part from (train data)
y_hat=decission_tree.predict(X_test)

# to check if there is overfitting or no)
print("prediction on training data :",f1_score(y_train, y_hat2, average='micro'))
print("prediction on test part from training data :",f1_score(y_test, y_hat, average='micro'))
# fit all train data
decission_tree.fit(X,y)
# make prediction using test file to make our prediction 
y_hat=decission_tree.predict(test_data)
y_hat

prediction on training data : 0.8385360602798708
prediction on test part from training data : 0.75


array([5., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 3., 5., 4., 5., 3., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 3., 4., 4., 4., 5., 4., 4., 4., 4.,
       5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3.,
       4., 4., 4., 4., 5., 5., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 4., 4.

In [36]:
# count how many(1,2,3,4,5) in (y_hat)
un, co = np.unique(y_hat, return_counts= True)
print(un, co )

[3. 4. 5.] [ 17 429  33]


In [37]:
# to record the result on file to upload on Kaggle
pred_df = pd.DataFrame(data={'id': np.asarray(test_data.index), 'rating': y_hat})
pred_df.to_csv('pred_decission_2.csv', index=False)

**My question here  is how to tune a Decision Tree. What should be the range of values I should try for the maximum depth, which criteria shoulld I use ("gini","entropy")
which strategy  should used to choose the split at each node.(“best” ,“random” )?**

To respond to these questions, I need to know how changing one will influence my model. What does the criterion (gini or entropy) mean when it comes to increasing the maximum depth? I used Information Gain, which uses the entropy measure because it gives the most amount of information, and I assumed that entropy would improve the model's ability to learn from information because it gives the most amount of information, but I found no impact when I replaced the default (gini criteria) with (entropy criteria). I expect that the splitting criteria I choose will not have a significant impact on my model's performance.

here we have many of features, so “best” splitter would be ideal because it will calculate the best features to split based on the impurity measure and use that to split the nodes, whereas if choose “random” we have a high chance of ending up with features that don’t really give  much information, which would lead to a more deeper less precise tree. 
when I used  "random" splitter it's effaect made the perfomance less  but "best" splitter its effect in performance was good 

in the previous model when i did not define (max_depth) the model choose the (defoult) whish is the maximum max_depth  that lead to make the model overfitted  ,To avoid this  overfitting, we should reduce the number for max_depth , So I let (max_depth=3) and I found that the model's (f1_score) on training data is not very high this mean that the model did not learn well and (f1_score) on testing part also low ,So I tried to increase (max_depth=6) and it gives me a good performance than (max_depth=3 ) or remain the defoult (max_depth) and it gives me 0.79916 on kaggle






#Naive Bayse Model

In [38]:
# import GaussianNB from sklearn
from sklearn.naive_bayes import GaussianNB

# define decission tree model using hyperparamter
params_NB = {'var_smoothing': np.logspace(0,-5, num=100)}
NB= GaussianNB()
naive_classifier= GridSearchCV(NB,params_NB)
# fit model with train data
naive_classifier.fit(X_train, y_train)

#print the var_smoothing value
print(naive_classifier.best_params_)

# predict test part from (train data)
y_hat=naive_classifier.predict(X_test)

# to check if there is overfitting or no)
print("prediction on test part from training data :",f1_score(y_test, y_hat, average='micro'))

# fit all train data
naive_classifier.fit(X,y)
# make prediction using test file to make our prediction 
y_hat=naive_classifier.predict(test_data)
y_hat

{'var_smoothing': 1.0}
prediction on test part from training data : 0.7073170731707317


array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

In [39]:
# to record the result on file to upload on Kaggle
pred_df = pd.DataFrame(data={'id': np.asarray(test_data.index), 'rating': y_hat})
pred_df.to_csv('pred_walkthrough.csv', index=False)

The Naive Bayes algorithm is a straightforward classification method.
Tuning the model's hyper-parameters is one of the first things we try to do to increase its performance. The Naive Bayes classifier, on the other hand, has a fairly small parameter set.
Because the number of classes is sometimes the only parameter depending on the implementation, hyper-parameter tuning is not a valid strategy for improving Naive Bayes classifier accuracy.
So, by applying some simple approaches to the dataset, such as data pretreatment, we may improve the Naive Bayes classifier.

**Naïve Bayes is a probabilistic classifier based on Bayes theorem** so we use Laplace smoothing  which is a smoothing technique to handles the problem of zero probability in Naïve Bayes. 

when I use naive model to make the classification it wasn't the best model 
because  (f1_score) wasn't high it was 0.66945

#SVM Model 1

In [40]:
#make scaler for train data
X_train=StandardScaler().fit_transform(X_train)
X_test=StandardScaler().fit_transform(X_test)
X=StandardScaler().fit_transform(X)
# make scaler for test data
test_data_sca=StandardScaler().fit_transform(test_data)

In [41]:
#imports
# import SVC model from sklearn.svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# define SVM model
svm=SVC(C=1000,kernel='rbf')

# fit model with train data
svm.fit(X_train, y_train)

# make prediction on training data to know how much the model learn from train data 
y_hat2=svm.predict(X_train)
# predict test part from (train data)
y_hat=svm.predict(X_test)

# to check if there is overfitting or no)
print("prediction on training data :",f1_score(y_train, y_hat2, average='micro'))
print("prediction on test part from training data :",f1_score(y_test, y_hat, average='micro'))

# make prediction using test file to make our prediction
svm.fit(X,y)
y_hat=svm.predict(test_data_sca)
y_hat

prediction on training data : 0.9989235737351991
prediction on test part from training data : 0.5853658536585366


array([5., 4., 3., 5., 4., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 3., 3., 4., 4., 5., 4., 4., 5., 4., 5., 4., 4., 5., 3., 4., 4.,
       3., 4., 4., 4., 4., 3., 5., 4., 4., 4., 4., 3., 3., 3., 4., 4., 4.,
       3., 3., 3., 4., 3., 4., 3., 3., 4., 4., 4., 4., 4., 5., 4., 4., 3.,
       4., 4., 3., 4., 4., 4., 3., 3., 4., 4., 4., 4., 5., 3., 4., 4., 3.,
       4., 4., 4., 4., 5., 5., 4., 4., 4., 4., 4., 5., 4., 4., 3., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 5., 3., 4., 4., 4., 4., 5., 3., 4., 5.,
       4., 3., 4., 3., 4., 4., 5., 4., 4., 5., 4., 4., 4., 3., 4., 2., 5.,
       4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       3., 3., 4., 3., 4., 5., 4., 4., 4., 3., 4., 3., 4., 4., 3., 4., 4.,
       4., 4., 4., 3., 2., 4., 4., 4., 4., 4., 5., 4., 3., 4., 3., 4., 4.,
       4., 4., 3., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 4., 5., 3., 4., 3., 5., 4., 5., 4., 4., 4., 4., 5., 4.,
       4., 5., 4., 4., 4.

In [42]:
# count how many(1,2,3,4,5) in (y_hat)
un, co = np.unique(y_hat, return_counts= True)
print(un, co )

[2. 3. 4. 5.] [  4  86 325  64]


In [43]:
# to record the result on file to upload on Kaggle
pred_df = pd.DataFrame(data={'id': np.asarray(test_data.index), 'rating': y_hat})
pred_df.to_csv('pred_walkthrough.csv', index=False)

**C parameterpr** the penalty parameter denotes the phrase misclassification or error. The SVM optimisation uses the misclassification or error term to determine how much mistake is tolerable.
When C is high, all data points will be accurately classified; nevertheless, there is a risk of overfitting.
**here** in my model when define C with large number the (f1_score) in the prediction of training data part was very high and (f1_score) in the prediction of testing part of data was very low which model may overfitted and (f1_score on kaggle was not high it was 0.65690

#SVM Model 2

In [44]:
#imports
# import SVC model from sklearn.svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# define  hyperparamter for SVM model
param_grid = { 'C':[1,4,5,10,15],'kernel':['rbf']}
# define SVM model
svm=SVC()
classifier2= GridSearchCV(svm, param_grid, refit = True, verbose = 3)
# fit model with train data
classifier2.fit(X_train, y_train)
# print hyperparamter
print("**************************************")
print(classifier2.best_params_)
print("**************************************")

# make prediction on training data to know how much the model learn from train data 
y_hat2=classifier2.predict(X_train)
# predict test part from (train data)
y_hat=classifier2.predict(X_test)

# to check if there is overfitting or no)
print("prediction on training data :",f1_score(y_train, y_hat2, average='micro'))
print("prediction on test part from training data :",f1_score(y_test, y_hat, average='micro'))

# make prediction using test file to make our prediction
classifier2.fit(X,y)

y_hat=classifier2.predict(test_data_sca)
y_hat

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ...................C=1, kernel=rbf;, score=0.720 total time=   0.2s
[CV 2/5] END ...................C=1, kernel=rbf;, score=0.715 total time=   0.1s
[CV 3/5] END ...................C=1, kernel=rbf;, score=0.742 total time=   0.1s
[CV 4/5] END ...................C=1, kernel=rbf;, score=0.720 total time=   0.1s
[CV 5/5] END ...................C=1, kernel=rbf;, score=0.708 total time=   0.1s
[CV 1/5] END ...................C=4, kernel=rbf;, score=0.710 total time=   0.1s
[CV 2/5] END ...................C=4, kernel=rbf;, score=0.715 total time=   0.1s
[CV 3/5] END ...................C=4, kernel=rbf;, score=0.726 total time=   0.1s
[CV 4/5] END ...................C=4, kernel=rbf;, score=0.726 total time=   0.1s
[CV 5/5] END ...................C=4, kernel=rbf;, score=0.730 total time=   0.1s
[CV 1/5] END ...................C=5, kernel=rbf;, score=0.715 total time=   0.1s
[CV 2/5] END ...................C=5, kernel=rbf;,

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

In [46]:
# to record the result on file to upload on Kaggle
pred_df = pd.DataFrame(data={'id': np.asarray(test_data.index), 'rating': y_hat})
pred_df.to_csv('pred_walkthrough2.csv', index=False)

C parameterpr the penalty parameter denotes the phrase misclassification or error. The SVM optimisation uses the misclassification or error term to determine how much mistake is tolerable. 
 here in my model when define C with small number (C=5) the (f1_score) in the prediction of training data part was about 0.83 and (f1_score) in the prediction of testing part of data was about 0.71 
and gives me 0.75313 on Kaggle 
I used GridSearch to get the best hyperparameter (the best C) to get the higher performance

#Problem Formulation:




**Define The problem**

The problem is that We have  a data from wish.com
the data contain some available data ,nosies and mising value which need to make preprocessing on it 
and we want to pridect the ratings which  are in categories from 1 to 5.

**inputs**

our inputs are some features from wish.com which use to help us to get Rating Prediction 

**outputs**

our output is the product's rating from 1 to 5 

**What data mining function is requride?**

 we use classification function (decision tree classifier, naive bayes classifier , SVM classifier) models to get Rating Prediction of our product

 **What could be the challenges?**



1.   The challenge is that we need to make preprocessing in our data
2.   fill nun values, remove outliers,drop columns that are useless and have not an impact in data 

1.   choose the best hyper paramter in model to get high (f1_score) accuracy

**What is the impact?**

When Wish.com be able to classify the products rating itsel befor peaple doing  by using models .in this case Wish.com can offer products that have high rating which customers need  and this will increase sells 

**What is an ideal solution?**

The ideal solution is that, make the model learn well and could classify the product in a right way via making a good preprocessing for the given data and insertion these data to classification models to make a right classification for the product
I tried to make classification using (Decission tree , SVm and Naive bayse) and find the best of them is Decission tree because it gives me the higher (f1_score)

















#Questions nswers

**Why Data Mining is a misnomer? What is another preferred name?**

Datab Data Mining is a misnomer because we did not mining for the data because we get the data from collection process 
but we mining for the data so the  proper name is knowledge descovry from the data.

another preferred name for data mining is Knowledge discovery (mining) in databases (KDD), 
knowledge extraction, data analysis, data archeology, data dredging, information harvesting,business intelligence

**What is the general knowledge discovery process? What is the difference between a data engineer and data scientist/AI engineer?**

the general knowledge discovery proces is 

**1-Understanding application domain**

**2-Selection:**choose a data set, or focus on a subset of variables

**3-preprocessing and cleaning:**
    Noise and outliers are removed.
    Getting the data you need to model or account for noise.
    Methods for dealing with data fields that are missing.

**4-Transformation:**
    Depending on the task's purpose, choose suitable characteristics to represent the data.
    reduce the effective number of variables under consideration or to find invariant representations for the data by Using dimensionality reduction or transformation methods

**5-Data Mining:**
which are an algorithms used to extract significant patterns from altered data, which aid in the development of prediction models.

**6-Evaluation:**
It is the process of recognising the increasing patterns that indicate knowledge based on predetermined metrics.

**7-Knowledge representation**:

This is the final phase in the KDD process, and it demands that the 'knowledge' retrieved in the previous step be applied to a specific application
or domain in a visual format such as tables,reports, and so on

he difference between a data engineer and data scientist/AI engineer is

**Data engineers:** are data specialists who provide big data infrastructure for data scientists to make analysis

**Data scientist :** The job of a data scientist is to conversion raw data into cleverness  and knowledge.
They dealing with scientific challenges using statistics, machine learning, and analytic methodologies.

**in data mining, what is the difference between prediction and categorization?**

categorization: is The process of determining the category or class label to which a new observation belongs to

prediction:is the process of recognising the numerical data that is absent or unavailable for a new observation.
Prediction is unaffected by the class label,

**Why data science/machine learning is a bad idea in the context of information security?** 

because it increase uncertainty and increase risk of data breach and fine
Difficult to evaluate change in AI
Difficult to verify against compliance

**What is CIA principle and how can we use it to access the security/privacy aspect of the AI system/pipelines?**

CIA  is stand for confidentiality, integrity, and availability 
these three principles represent very importance for  any organization’s security infrastructure
so they should be goals and objectives for every security program.

confidentiality: suitable procedures to secure sensitive data and maintain management

integrity: method is used to stop losing data or being altered 

availability: information can be reached easily when individuals need it 
Applying the  CIA Principles is depending on an organization’s security goals and the nature of the business,and any relevant regulatory constraints.


